# Baseline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

# Load your dataset
data = pd.read_csv('your_dataset.csv')
# Assuming the columns are named 'Date' and 'Y'
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Perform any necessary data transformations here, if needed
# For example, you can take the log of 'Y' if the data is not stationary
# data['Y'] = np.log(data['Y'])

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:]

# Fit an ARIMA model
order = (p, d, q)  # Define your order (p, d, q) values
model = ARIMA(train, order=order)
model_fit = model.fit(disp=0)

# Make predictions
predictions = model_fit.forecast(steps=len(test))

# Calculate RMSE (Root Mean Squared Error) to evaluate the model
rmse = np.sqrt(mean_squared_error(test, predictions))
print(f'RMSE: {rmse}')

# Plot the original data and predictions
plt.figure(figsize=(12, 6))
plt.plot(test.index, test['Y'], label='Actual')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.legend()
plt.title('ARIMA Forecast')
plt.show()


ModuleNotFoundError: No module named 'darts'